# English-French Translation based on Tensorflow

# 1 - Import packages

In [36]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm

%matplotlib inline

In [4]:
tf.__version__

'1.9.0'

## Parallel Corpus Loading and Data Analysis

In [6]:
# English source data
with open("data/small_vocab_en", "r", encoding="utf-8") as f:
    source_text = f.read()

# French target data
with open("data/small_vocab_fr", "r", encoding="utf-8") as f:
    target_text = f.read()

In [7]:
source_text[:500]

'new jersey is sometimes quiet during autumn , and it is snowy in april .\nthe united states is usually chilly during july , and it is usually freezing in november .\ncalifornia is usually quiet during march , and it is usually hot in june .\nthe united states is sometimes mild during june , and it is cold in september .\nyour least liked fruit is the grape , but my least liked is the apple .\nhis favorite fruit is the orange , but my favorite is the grape .\nparis is relaxing during december , but it '

In [8]:
target_text[0:500]

"new jersey est parfois calme pendant l' automne , et il est neigeux en avril .\nles états-unis est généralement froid en juillet , et il gèle habituellement en novembre .\ncalifornia est généralement calme en mars , et il est généralement chaud en juin .\nles états-unis est parfois légère en juin , et il fait froid en septembre .\nvotre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .\nson fruit préféré est l'orange , mais mon préféré est le raisin .\nparis est relaxant en décembre "

In [8]:
view_sentence_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

print("-"*5 + "English Text" + "-"*5)
sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))
print('Max number of words in a sentence: {}'.format(np.max(word_counts)))

print()
print("-"*5 + "French Text" + "-"*5)
sentences = target_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))
print('Max number of words in a sentence: {}'.format(np.max(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
-----English Text-----
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028
Max number of words in a sentence: 17

-----French Text-----
Number of sentences: 137861
Average number of words in a sentence: 14.226612312401622
Max number of words in a sentence: 23

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the l

## Data Preprocessing
#### 'I love machine learning and deep learning' is transfer to [28, 29, 274, 873, 12, 983, 873]

In [27]:
# Build up a English dic
source_vocab = list(set(source_text.lower().split()))
# Build up a French dic
target_vocab = list(set(target_text.lower().split()))

In [28]:
print("The size of English vocab is : {}".format(len(source_vocab)))
print("The size of French vocab is : {}".format(len(target_vocab)))

The size of English vocab is : 227
The size of French vocab is : 354


In [29]:
# Special character
SOURCE_CODES = ['<PAD>', '<UNK>']
TARGET_CODES = ['<PAD>', '<EOS>', '<UNK>', '<GO>']

In [30]:
source_vocab_to_int = {word: idx for idx, word in enumerate(SOURCE_CODES + source_vocab)}
source_int_to_vocab = {idx: word for idx, word in enumerate(SOURCE_CODES + source_vocab)}

target_vocab_to_int = {word: idx for idx, word in enumerate(TARGET_CODES + target_vocab)}
target_int_to_vocab = {idx: word for idx, word in enumerate(TARGET_CODES + target_vocab)}

In [31]:
source_vocab_to_int['dry']

7

In [32]:
source_int_to_vocab[7]

'dry'

In [33]:
print("The size of English Map is : {}".format(len(source_vocab_to_int)))
print("The size of French Map is : {}".format(len(target_vocab_to_int)))

The size of English Map is : 229
The size of French Map is : 358


## 2.4 - Corpus Transformation

We can now use the dics above to perform coupus transformation. Since LSTM can only handle fix-size input data, we need to maintain the length of input corpus and output corpus. Let Tx = 20, sentences longer than 20 will be cut down

”I love machine learning and deep learning"，is transfer to [28, 29, 274, 873, 12, 983, 873, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
def text_to_int(sentence, map_dict, max_length=20, is_target=False):
    """
    
    @param sentence: 
    @param map_dict: vocab to dict
    @param max_length: max sentence length
    @param is_target: if is target sentence, we need to add <EOS> behind the sentence
    """
    
    text_to_idx = []
    # unk index
    unk_idx = map_dict.get("<UNK>")
    pad_idx = map_dict.get("<PAD>")
    eos_idx = map_dict.get("<EOS>")
    
    if not is_target:
        for word in sentence.lower().split():
            text_to_idx.append(map_dict.get(word, unk_idx))
    
    else:
        for word in sentence.lower().split():
            text_to_idx.append(map_dict.get(word, unk_idx))
        text_to_idx.append(eos_idx)
    
    # cut down
    if len(text_to_idx) > max_length:
        return text_to_idx[:max_length]
    
    # shorter than 20 add<PAD>
    else:
        text_to_idx = text_to_idx + [pad_idx] * (max_length - len(text_to_idx))
        return text_to_idx

In [37]:
# Transformation of English sentences
source_text_to_int = []

for sentence in tqdm.tqdm(source_text.split("\n")):
    source_text_to_int.append(text_to_int(sentence, source_vocab_to_int, 20, 
                                          is_target=False))

100%|██████████| 137861/137861 [00:02<00:00, 48551.29it/s]


In [38]:
# Transformation of French sentences
target_text_to_int = []

for sentence in tqdm.tqdm(target_text.split("\n")):
    target_text_to_int.append(text_to_int(sentence, target_vocab_to_int, 25, 
                                          is_target=True))

100%|██████████| 137861/137861 [00:03<00:00, 37772.26it/s]


In [39]:
random_index = 77

print("-"*5 + "English example" + "-"*5)
print(source_text.split("\n")[random_index])
print(source_text_to_int[random_index])

print()
print("-"*5 + "French example" + "-"*5)
print(target_text.split("\n")[random_index])
print(target_text_to_int[random_index])

-----English example-----
the united states is never beautiful during march , and it is usually relaxing in summer .
[168, 48, 71, 80, 204, 192, 158, 100, 36, 218, 130, 80, 96, 22, 206, 159, 126, 0, 0, 0]

-----French example-----
les états-unis est jamais belle en mars , et il est relaxant habituellement en été .
[357, 333, 223, 116, 327, 294, 48, 276, 45, 299, 223, 111, 307, 294, 31, 225, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [40]:
X = np.array(source_text_to_int)
Y = np.array(target_text_to_int)

In [41]:
X

array([[ 77, 201,  80, ...,   0,   0,   0],
       [168,  48,  71, ...,   0,   0,   0],
       [170,  80,  96, ...,   0,   0,   0],
       ...,
       [182,  80, 204, ...,   0,   0,   0],
       [168,  32,  80, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

#  Building Model

- model_inputs
- Encoder, encoder_layer
- Decoder
    - decoder_layer_inputs
    - decoder_layer_train
    - decoder_layer_inference(inference and prediction)
- Final Seq2Seq Model

## B.1 - model input

In [42]:
#Initialize some tensors, used as model input
def model_inputs():
    """
    
    return：inputs, targets, learning_rate, source_sequence_len, target_sequence_len, max_target_sequence_len
    """
    inputs = tf.placeholder(tf.int32, [None, None], name="inputs")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    
    source_sequence_len = tf.placeholder(tf.int32, (None,), name="source_sequence_len")
    target_sequence_len = tf.placeholder(tf.int32, (None,), name="target_sequence_len")
    max_target_sequence_len = tf.placeholder(tf.int32, (None,), name="max_target_sequence_len")
    
    return inputs, targets, learning_rate, source_sequence_len, target_sequence_len, max_target_sequence_len

## B.2 - Encoder

In [44]:
# # Build RNN cell
# encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

# # Run Dynamic RNN
# #   encoder_outputs: [max_time, batch_size, num_units]
# #   encoder_state: [batch_size, num_units]
# encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
#     encoder_cell, encoder_emb_inp,
#     sequence_length=source_sequence_length, time_major=True)



def encoder_layer(rnn_inputs, rnn_size, rnn_num_layers,
                  source_sequence_len, source_vocab_size, encoder_embedding_size=100):
    """
    Build up the Encoder
    
    @param rnn_inputs: input of rnn
    @param rnn_size: the number of node in rnn
    @param rnn_num_layers: the number of layers of rnn
    @param source_sequence_len
    @param source_vocab_size
    @param encoder_embedding_size: the size of vector after embeding for each words 
    """
    # For the vocab in input sentences, do embeding: https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence
    # 
    encoder_embed = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoder_embedding_size)
    
    # LSTM Cell
    def get_lstm(rnn_size):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=123))
        return lstm
    
    # multi layer LSTM
    lstms = tf.contrib.rnn.MultiRNNCell([get_lstm(rnn_size) for _ in range(rnn_num_layers)])
    encoder_outputs, encoder_states = tf.nn.dynamic_rnn(lstms, encoder_embed, source_sequence_len, 
                                                        dtype=tf.float32)
    
    return encoder_outputs, encoder_states

## B.3 - Decoder

### B.3.1 - Decoder Layer Inputs

In [45]:
def decoder_layer_inputs(target_data, target_vocab_to_int, batch_size):
    """
    Preprocessing towards the input of decoder
    
    @param target_data: the tensor of French 
    @param target_vocab_to_int
    @param batch_size: batch size
    """
    # one batch has several sentences, eliminate last word in each sentences
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])#https://blog.csdn.net/banana1006034246/article/details/75092388
    # add”<GO>" in the front of every input sentences
    decoder_inputs = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int["<GO>"]), 
                                ending], 1)
    
    return decoder_inputs

### B.3.2 - Decoder Traing

#### Here, implement Teacher Forcing to optimize training 

In [46]:
def decoder_layer_train(encoder_states, decoder_cell, decoder_embed,
                        target_sequence_len, max_target_sequence_len, output_layer):
    
    """
    
    
    @param encoder_states: Context Vector from encoder
    @param decoder_cell: Decoder cell that will build in B.3.4 
    @param decoder_embed: embing vector of the decoder input word
    @param target_sequence_len
    @param max_target_sequence_len
    @param output_layer
    """
    
    # An object used to form training decoder
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embed,
                                                       sequence_length=target_sequence_len,
                                                       time_major=False)
    
    training_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,
                                                      training_helper,
                                                      encoder_states,
                                                      output_layer)
    
    training_decoder_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                  impute_finished=True,
                                                                  maximum_iterations=max_target_sequence_len)
    #Python boolean. If True, then states for batch entries which are marked as 
    #finished get copied through and the corresponding outputs get zeroed out.
    
    return training_decoder_outputs

### B.3.3 - Decoder Inference

In [47]:
def decoder_layer_infer(encoder_states, decoder_cell, decoder_embed, start_id, end_id, 
                        max_target_sequence_len, output_layer, batch_size):
    """
    Decoder prediction
    
    @param encoder_states: Context Vector from encoder
    @param decoder_cell: Decoder cell that will build in B.3.4
    @param decoder_embed: embing vector of the decoder input word
    @param start_id: the token id of start word in a sentence，"<GO>"
    @param end_id: the token id of end word in a sentence，，"<EOS>"
    @param max_target_sequence_len
    @param output_layer
    @batch_size: batch size
    """
    #tf.tile used as tensor expansion
    start_tokens = tf.tile(tf.constant([start_id], dtype=tf.int32), [batch_size], name="start_tokens")#https://blog.csdn.net/loseinvain/article/details/78994615
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embed,
                                                                start_tokens,
                                                                end_id) # similar with tf.contrib.seq2seq.TrainingHelper
    
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,
                                                       inference_helper,
                                                       encoder_states,
                                                       output_layer)
    
    inference_decoder_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                      impute_finished=True,
                                                                      maximum_iterations=max_target_sequence_len)
    
    return inference_decoder_outputs
    

### B.3.4 - Decoder Layer

In [48]:
# The integration of those three function above
def decoder_layer(encoder_states, decoder_inputs, target_sequence_len, 
                   max_target_sequence_len, rnn_size, rnn_num_layers,
                   target_vocab_to_int, target_vocab_size, decoder_embedding_size, batch_size):
    """
    Build up final decoder    
        
    @param encoder_states: Context Vector from encoder
    @param decoder_inputs
    @param target_sequence_len
    @param max_target_sequence_len
    @param rnn_size: number of nodes in rnn cell
    @param rnn_num_layers
    @param target_vocab_to_int
    @param target_vocab_size
    @param decoder_embedding_size
    @param batch_size
    """
    #do embedding to the decoder_inputs( <Go> I like machine learning )
    decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoder_embedding_size]))
    decoder_embed = tf.nn.embedding_lookup(decoder_embeddings, decoder_inputs)
    
    def get_lstm(rnn_size):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=456))
        return lstm#build up lstmCell for one layer
    
    decoder_cell = tf.contrib.rnn.MultiRNNCell([get_lstm(rnn_size) for _ in range(rnn_num_layers)])
    
    # output_layer logits
    output_layer = tf.layers.Dense(target_vocab_size)
    
    with tf.variable_scope("decoder"):
        training_logits = decoder_layer_train(encoder_states,
                                               decoder_cell,
                                               decoder_embed,
                                               target_sequence_len,
                                               max_target_sequence_len,
                                               output_layer)
    
    with tf.variable_scope("decoder", reuse=True):
        inference_logits = decoder_layer_infer(encoder_states,
                                               decoder_cell,
                                               decoder_embeddings,
                                               target_vocab_to_int["<GO>"],
                                               target_vocab_to_int["<EOS>"],
                                                max_target_sequence_len,
                                                output_layer,
                                                batch_size)#use embeddings because when infer, we do not know real label
    
    return training_logits, inference_logits

## 3.3 - Seq2Seq模型

In [49]:
def seq2seq_model(input_data, target_data, batch_size,
                 source_sequence_len, target_sequence_len, max_target_sentence_len,
                 source_vocab_size, target_vocab_size,
                 encoder_embedding_size, decoder_embeding_size,
                 rnn_size, rnn_num_layers, target_vocab_to_int):
    
    """
    Build up Seq2Seq by integrate previous encoder and decoder
    
    @param input_data: tensor of input data
    @param target_data: tensor of target data
    @param batch_size: batch size
    @param source_sequence_len
    @param target_sequence_len
    @param max_target_sentence_len
    @param source_vocab_size
    @param target_vocab_size
    @param encoder_embedding_size
    @param decoder_embedding_size
    @param rnn_size: number of nodes in rnn cell
    @param rnn_num_layers
    @param target_vocab_to_int
    """
    _, encoder_states = encoder_layer(input_data, rnn_size, rnn_num_layers, source_sequence_len, 
                                      source_vocab_size, encoder_embedding_size)
    
    decoder_inputs = decoder_layer_inputs(target_data, target_vocab_to_int, batch_size)
    
    training_decoder_outputs, inference_decoder_outputs = decoder_layer(encoder_states,
                                                                       decoder_inputs,
                                                                      target_sequence_len,
                                                                       max_target_sentence_len,
                                                                      rnn_size,
                                                                      rnn_num_layers,
                                                                      target_vocab_to_int,
                                                                      target_vocab_size,
                                                                      decoder_embeding_size,
                                                                       batch_size)
    return training_decoder_outputs, inference_decoder_outputs

# Training Model

## T.1 - Parameter

In [51]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 128
# Number of Layers
rnn_num_layers = 1
# Embedding Size
encoder_embedding_size = 100
decoder_embedding_size = 100
# Learning Rate
lr = 0.001
# Print out cycle
display_step = 50

## T.2 - Build Graph

In [52]:

train_graph = tf.Graph()

with train_graph.as_default():
    inputs, targets, learning_rate, source_sequence_len, target_sequence_len, _ = model_inputs()
    
    max_target_sequence_len = 25
    train_logits, inference_logits = seq2seq_model(tf.reverse(inputs, [-1]),
                                                  targets,
                                                  batch_size,
                                                  source_sequence_len,
                                                  target_sequence_len,
                                                  max_target_sequence_len,
                                                  len(source_vocab_to_int),
                                                  len(target_vocab_to_int),
                                                  encoder_embedding_size,
                                                  decoder_embedding_size,
                                                  rnn_size,
                                                  rnn_num_layers,
                                                  target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name="logits")#identity used to copy a same tensor as input tensor 
    inference_logits = tf.identity(inference_logits.sample_id, name="predictions")
    
    masks = tf.sequence_mask(target_sequence_len, max_target_sequence_len, dtype=tf.float32, name="masks")
#     tf.sequence_mask([1, 3, 2], 5)  # [[True, False, False, False, False],
#                                 #  [True, True, True, False, False],
#                                 #  [True, True, False, False, False]]
    
    #gradient descent 
    with tf.name_scope("optimization"):
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks)
        
        optimizer = tf.train.AdamOptimizer(learning_rate)
        
        gradients = optimizer.compute_gradients(cost)
#       Given a tensor t, this operation returns a tensor of the same type and shape as t with 
#       its values clipped to clip_value_min and clip_value_max. Any values less than clip_value_min 
#       are set to clip_value_min. Any values greater than clip_value_max are set to clip_value_max.
        clipped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(clipped_gradients)

## B.3 - Train Model

In [52]:
def get_batches(sources, targets, batch_size):

    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]
        
        # Need the lengths for the _lengths parameters
        targets_lengths = []
        for target in targets_batch:
            targets_lengths.append(len(target))

        source_lengths = []
        for source in sources_batch:
            source_lengths.append(len(source))

        yield sources_batch, targets_batch, source_lengths, targets_lengths

In [54]:
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(source_text_to_int, target_text_to_int, batch_size)):
    
            _, loss = sess.run(
                [train_op, cost],
                {inputs: source_batch,
                 targets: target_batch,
                 learning_rate: lr,
                 source_sequence_len: sources_lengths,
                 target_sequence_len: targets_lengths})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {inputs: source_batch,
                     source_sequence_len: sources_lengths,
                     target_sequence_len: targets_lengths})

                print('Epoch {:>3} Batch {:>4}/{} - Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_text_to_int) // batch_size, loss))
        
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, "checkpoints/dev")
    print('Model Trained and Saved')

Epoch   0 Batch   50/1077 - Loss: 2.4797
Epoch   0 Batch  100/1077 - Loss: 2.0935
Epoch   0 Batch  150/1077 - Loss: 1.7045
Epoch   0 Batch  200/1077 - Loss: 1.3225
Epoch   0 Batch  250/1077 - Loss: 1.0670
Epoch   0 Batch  300/1077 - Loss: 0.8785
Epoch   0 Batch  350/1077 - Loss: 0.8020
Epoch   0 Batch  400/1077 - Loss: 0.7552
Epoch   0 Batch  450/1077 - Loss: 0.7071
Epoch   0 Batch  500/1077 - Loss: 0.6214
Epoch   0 Batch  550/1077 - Loss: 0.6254
Epoch   0 Batch  600/1077 - Loss: 0.5923
Epoch   0 Batch  650/1077 - Loss: 0.5505
Epoch   0 Batch  700/1077 - Loss: 0.4917
Epoch   0 Batch  750/1077 - Loss: 0.4815
Epoch   0 Batch  800/1077 - Loss: 0.4748
Epoch   0 Batch  850/1077 - Loss: 0.3945
Epoch   0 Batch  900/1077 - Loss: 0.4011
Epoch   0 Batch  950/1077 - Loss: 0.3432
Epoch   0 Batch 1000/1077 - Loss: 0.3491
Epoch   0 Batch 1050/1077 - Loss: 0.3317
Epoch   1 Batch   50/1077 - Loss: 0.3067
Epoch   1 Batch  100/1077 - Loss: 0.2766
Epoch   1 Batch  150/1077 - Loss: 0.2458
Epoch   1 Batch 

Epoch   9 Batch  600/1077 - Loss: 0.0149
Epoch   9 Batch  650/1077 - Loss: 0.0069
Epoch   9 Batch  700/1077 - Loss: 0.0065
Epoch   9 Batch  750/1077 - Loss: 0.0073
Epoch   9 Batch  800/1077 - Loss: 0.0123
Epoch   9 Batch  850/1077 - Loss: 0.0082
Epoch   9 Batch  900/1077 - Loss: 0.0109
Epoch   9 Batch  950/1077 - Loss: 0.0104
Epoch   9 Batch 1000/1077 - Loss: 0.0119
Epoch   9 Batch 1050/1077 - Loss: 0.0087
Model Trained and Saved


# Prediction

In [53]:
def sentence_to_seq(sentence, source_vocab_to_int):
    """
    Transform sentences to vector
    """
    unk_idx = source_vocab_to_int["<UNK>"]
    word_idx = [source_vocab_to_int.get(word, unk_idx) for word in sentence.lower().split()]
    
    return word_idx

In [ ]:
translate_sentence_text = input("Please Enter Sentence：")

In [57]:
translate_sentence = sentence_to_seq(translate_sentence_text, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph('checkpoints/dev.meta')
    loader.restore(sess, tf.train.latest_checkpoint('./checkpoints'))

    input_data = loaded_graph.get_tensor_by_name('inputs:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_len:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_len:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size})[0]

print('【Input】')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\n【Prediction】')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in translate_logits]))

print("\n【Full Sentence】")
print(" ".join([target_int_to_vocab[i] for i in translate_logits]))

INFO:tensorflow:Restoring parameters from ./checkpoints/dev
【Input】
  Word Ids:      [129, 221, 155, 192, 55, 8, 197, 103, 170, 105, 192, 168, 80, 137, 209]
  English Words: ['the', 'united', 'states', 'is', 'busy', 'during', 'spring', ',', 'and', 'it', 'is', 'usually', 'freezing', 'in', 'november']

【Prediction】
  Word Ids:      [205, 123, 67, 274, 44, 193, 310, 201, 311, 67, 101, 171, 227, 254, 37, 1]
  French Words: ['les', 'états-unis', 'est', 'occupé', 'au', 'printemps', ',', 'et', 'il', 'est', 'généralement', 'pluvieux', 'en', 'novembre', '.', '<EOS>']

【Full Sentence】
les états-unis est occupé au printemps , et il est généralement pluvieux en novembre . <EOS>


##### the united states is busy in the spring, and it is usually rainy in november